# Checking Cuda Environment

In [1]:
import torch
import os

# Confirm that the GPU is detected

assert torch.cuda.is_available()

# Get the GPU device name.
device_name = torch.cuda.get_device_name()
n_gpu = torch.cuda.device_count()
print(f"Found device: {device_name}, n_gpu: {n_gpu}")
print(f'Cuda version: {torch.version.cuda}')
device = torch.device("cuda")
print(f'n_cpu: {os.cpu_count()}')

Found device: RTX A5000, n_gpu: 1
Cuda version: 11.7
n_cpu: 32


# Loading Pre-trained GhostNet

In [2]:
import torch
from ghostnet import ghostnet

model = ghostnet(num_classes=1000, width=1.0, dropout=0.2)
input = torch.randn(32,3,224,224)
y = model(input)
print(y)

tensor([[ 0.1238, -0.1265, -0.0423,  ..., -0.1444, -0.1636,  0.0296],
        [ 0.0653, -0.1379, -0.0777,  ..., -0.2142, -0.1265,  0.0395],
        [ 0.1401, -0.1992, -0.0970,  ..., -0.1884, -0.0956, -0.0643],
        ...,
        [ 0.1050, -0.1748, -0.0276,  ..., -0.1166, -0.1524,  0.0597],
        [ 0.1194, -0.0953, -0.0434,  ..., -0.0666, -0.1332, -0.0068],
        [ 0.0598, -0.2501, -0.0839,  ..., -0.1172, -0.1511, -0.0081]],
       grad_fn=<AddmmBackward0>)


In [3]:
model.load_state_dict(torch.load('state_dict_73.98.pth'))
model = torch.nn.DataParallel(model, device_ids = list(range(n_gpu))).cuda()

In [4]:
from PIL import Image
from torchvision import transforms
import torch

imsize = 224
loader = transforms.Compose([transforms.Resize(imsize), transforms.ToTensor()])


def load_image(img_name):
    image = Image.open(img_name)
    image = loader(image).float()
    image = image.clone().detach().requires_grad_(True)
    image = image.unsqueeze(0)
    image = image.cuda()
    return image

# Tested on images from https://github.com/EliSchwartz/imagenet-sample-images
images = [load_image('ImageNetData/{0}.JPEG'.format(i)) for i in ['black_widow', 'magpie_0', 'magpie_1']]
image = images[0]
pred = model(image)
_, indices = torch.topk(pred, k=15)
print(indices)

tensor([[ 75, 622, 527, 763, 495, 526, 120, 926, 770, 155, 935, 449, 445,  72,
         476]], device='cuda:0')


# Changing Last Layer

In [5]:
import torch.nn as nn

num_classes = 8
num_features = model.module.classifier.in_features
new_last_layer = nn.Linear(num_features, num_classes).cuda()
model.module.classifier = new_last_layer

# Fine Tuning Model

In [6]:
from timm.data import create_dataset

dataset = create_dataset(name='', root='AffectNetData', transform=loader)
image, label = dataset[10000]
image.shape

torch.Size([3, 224, 224])

In [7]:
import torch
from torch.utils.data import DataLoader

batch_size = 64

train_size = int(0.8 * len(dataset))  # 80% for training
val_size = int(0.1 * len(dataset))   # 10% for validation
test_size = len(dataset) - train_size - val_size  # Remaining 10% for test

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    dataset, [train_size, val_size, test_size])

num_workers = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, num_workers=num_workers)
test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=num_workers)

In [ ]:
from tqdm import tqdm
import copy
# import warnings 

# warnings.filterwarnings("ignore")

num_epochs = 50
loss_fn = torch.nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.RMSprop(filter(lambda p: p.requires_grad, model.parameters()), lr=0.00001, weight_decay=0.01)
# optimizer=torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)

best_acc=0
best_model=None

for epoch in range(num_epochs):
    model.train()
    total_samples = 0
    correct_predictions = 0
    total_loss = 0
    
    loop = tqdm(train_loader)
    for images, labels in loop:
        optimizer.zero_grad()

        outputs = model(images)
        _, predicted = torch.max(outputs, dim=1)
        loss = loss_fn(outputs, labels.cuda())
        
        total_samples += labels.size(0)
        correct_predictions += (predicted == labels.cuda()).sum().item()
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
        
        loop.set_description(f"Epoch [{epoch}/{num_epochs}]")
        loop.set_postfix(loss=loss.item(), acc=(predicted == labels.cuda()).sum().item()/labels.size(0))
        
    avg_loss = total_loss / len(train_loader)
    avg_acc = correct_predictions / total_samples
    print("TRAINING")
    print("loss=", avg_loss, ", accuracy=", avg_acc)
    
#     if epoch % 20 == 0:
#         torch.save(model.state_dict(), f'ghostnet_checkpoints/checkpoint_{epoch}.pth')
    
    total_samples = 0
    correct_predictions = 0
    total_loss = 0
    model.eval()
    with torch.no_grad():
        epoch_val_accuracy = 0
        epoch_val_loss = 0
        for images, labels in val_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs, dim=1)
            loss = loss_fn(outputs, labels.cuda())

            total_samples += labels.size(0)
            correct_predictions += (predicted == labels.cuda()).sum().item()
            total_loss += loss.item()

    avg_loss = total_loss / len(val_loader)
    avg_acc = correct_predictions / total_samples
    print("VALIDATION")
    print("loss=", avg_loss, ", accuracy=", avg_acc)

    if best_acc < avg_acc:
        best_acc = avg_acc
        best_model=copy.deepcopy(model.state_dict())
        torch.save(model.state_dict(), f'ghostnet_checkpoints/checkpoint_model.pth')

Epoch [0/50]: 100%|██████████| 364/364 [00:28<00:00, 12.67it/s, acc=0, loss=2.06]     

TRAINING
loss= 1.9807201689416236 , accuracy= 0.22648818490939612


VALIDATION
loss= 1.8732643308846846 , accuracy= 0.2606749311294766


Epoch [1/50]: 100%|██████████| 364/364 [00:28<00:00, 12.97it/s, acc=0, loss=1.98]     

TRAINING
loss= 1.7700521801854228 , accuracy= 0.3223432186975423


...

VALIDATION
loss= 0.8914048516232035 , accuracy= 0.6694214876033058


Epoch [31/50]: 100%|██████████| 364/364 [00:27<00:00, 13.04it/s, acc=0, loss=2.25]     

TRAINING
loss= 0.6492084209080581 , accuracy= 0.7664959325097921


VALIDATION
loss= 0.8956508053385693 , accuracy= 0.6773415977961432


Epoch [32/50]:  18%|█▊        | 65/364 [00:05<00:22, 13.25it/s, acc=0.844, loss=0.481]

# Testing Model

In [11]:
if best_model is not None:
    model.load_state_dict(best_model)
    print(f"Best Validation acc:{best_acc}")

    total_samples = 0
    correct_predictions = 0
    total_loss = 0
    model.eval()
    with torch.no_grad():
        epoch_val_accuracy = 0
        epoch_val_loss = 0
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs, dim=1)
            loss = loss_fn(outputs, labels.cuda())

            total_samples += labels.size(0)
            correct_predictions += (predicted == labels.cuda()).sum().item()
            total_loss += loss.item()

    avg_loss = total_loss / len(test_loader)
    avg_acc = correct_predictions / total_samples
    print("TESTING")
    print("loss=", avg_loss, ", accuracy=", avg_acc)
    
    torch.save(best_model, f'ghostnet_checkpoints/best_model_0.pth')
else:
    print(f"No best model Best acc:{best_acc}")

Best Validation acc:0.65633608815427
TESTING
loss= 1.0011567758477253 , accuracy= 0.6444061962134251


In [ ]:
# Replace all ReLU activations with PReLU activations
for name, module in model.named_modules():
    if isinstance(module, nn.ReLU):
        setattr(model, name, nn.PReLU())